# IMBALANCED DATASET ASSIGNMENT

apply oversampling or undersampling techniques random over and undersampling, tomek link, smot and class weighing to an imbalanced dataset with fewer classes. Train the model on balanced dataset and find the performances metrics (accuracy, F1 score and AUC) and compare which technique is improving model performance.

## IMPORTING THE REQUIRED LIBRARIES

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN
from imblearn.under_sampling import TomekLinks
from sklearn.utils.class_weight import compute_class_weight 
from sklearn.preprocessing import LabelEncoder

In [2]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN
from imblearn.under_sampling import TomekLinks
from sklearn.utils.class_weight import compute_class_weight 

## READING OUR CSV "WEATHERAUS"

In [3]:
DF_Aus=pd.read_csv("weatherAUS.csv")

## CHEKING THE DATA

In [4]:
DF_Aus.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [5]:
DF_Aus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145460 entries, 0 to 145459
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           145460 non-null  object 
 1   Location       145460 non-null  object 
 2   MinTemp        143975 non-null  float64
 3   MaxTemp        144199 non-null  float64
 4   Rainfall       142199 non-null  float64
 5   Evaporation    82670 non-null   float64
 6   Sunshine       75625 non-null   float64
 7   WindGustDir    135134 non-null  object 
 8   WindGustSpeed  135197 non-null  float64
 9   WindDir9am     134894 non-null  object 
 10  WindDir3pm     141232 non-null  object 
 11  WindSpeed9am   143693 non-null  float64
 12  WindSpeed3pm   142398 non-null  float64
 13  Humidity9am    142806 non-null  float64
 14  Humidity3pm    140953 non-null  float64
 15  Pressure9am    130395 non-null  float64
 16  Pressure3pm    130432 non-null  float64
 17  Cloud9am       89572 non-null

## DATA CLEANING

In [6]:
DF_Aus = DF_Aus.drop(columns=['Date'])
DF_Aus =DF_Aus.dropna()

In [7]:
DF_Aus.isnull().sum()

Location         0
MinTemp          0
MaxTemp          0
Rainfall         0
Evaporation      0
Sunshine         0
WindGustDir      0
WindGustSpeed    0
WindDir9am       0
WindDir3pm       0
WindSpeed9am     0
WindSpeed3pm     0
Humidity9am      0
Humidity3pm      0
Pressure9am      0
Pressure3pm      0
Cloud9am         0
Cloud3pm         0
Temp9am          0
Temp3pm          0
RainToday        0
RainTomorrow     0
dtype: int64

## LABEL ENCODING

In [8]:
Label_Enc = LabelEncoder()
DF_Aus['Location'] = Label_Enc.fit_transform(DF_Aus['Location'])
DF_Aus['WindGustDir'] = Label_Enc.fit_transform(DF_Aus['WindGustDir'])
DF_Aus['WindDir9am'] = Label_Enc.fit_transform(DF_Aus['WindDir9am'])
DF_Aus['WindDir3pm'] = Label_Enc.fit_transform(DF_Aus['WindDir3pm'])
DF_Aus['RainToday'] = Label_Enc.fit_transform(DF_Aus['RainToday'])
DF_Aus['RainTomorrow'] = Label_Enc.fit_transform(DF_Aus['RainTomorrow'])
DF_Aus['Evaporation'] = Label_Enc.fit_transform(DF_Aus['Evaporation'])
DF_Aus['Sunshine'] = Label_Enc.fit_transform(DF_Aus['Sunshine'])
DF_Aus['Cloud9am'] = Label_Enc.fit_transform(DF_Aus['Cloud9am'])
DF_Aus['Cloud3pm'] = Label_Enc.fit_transform(DF_Aus['Cloud3pm'])

## SPLITTING THE DATA INTO FEATUERES AND TARGET VARIABLE

In [9]:
X = DF_Aus.drop('RainToday', axis=1)
y = DF_Aus['RainToday']

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) 

In [10]:
from collections import Counter

## Random Under Sampling

In [11]:
print("Distribution of Original Class:", Counter(y_train))
RUS = RandomUnderSampler(random_state=42)
X_RUS, y_RUS = RUS.fit_resample(X_train, y_train) 
print("Original Class Distribution:", Counter(y_RUS))

Distribution of Original Class: Counter({0: 35177, 1: 9959})
Original Class Distribution: Counter({0: 9959, 1: 9959})


## Random Over Sampling

In [12]:
from imblearn.over_sampling import RandomOverSampler
print("Original class distribution:", Counter(y_train))
ROS = RandomOverSampler(random_state=42)
X_ROS, y_ROS = ROS.fit_resample(X_train, y_train) 
print("Original class distribution:", Counter(y_ROS))

Original class distribution: Counter({0: 35177, 1: 9959})
Original class distribution: Counter({0: 35177, 1: 35177})


## SMOTE(Synthetic Minority Oversampling Technique)

In [13]:
print("Original class distribution:", Counter(y_train))
Smote = SMOTE(random_state=42)
X_Smote, y_Smote = Smote.fit_resample(X_train, y_train)
print("Original class distribution:", Counter(y_Smote))

Original class distribution: Counter({0: 35177, 1: 9959})
Original class distribution: Counter({0: 35177, 1: 35177})


## Tomek Links

In [14]:
TL = TomekLinks()
X_TL, y_TL = TL.fit_resample(X_train, y_train) 

## Class Weights

In [15]:
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = dict(enumerate(class_weights)) 
class_weights

array([0.64155556, 2.26609097])

## EVALUATING THE MODEL PERFORMACE

In [16]:
def evaluate_model(X_train, y_train, X_test, y_test, class_weights=None):
    model = RandomForestClassifier(class_weight=class_weights, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]
    
    print(classification_report(y_test, y_pred))
    print("AUC:", roc_auc_score(y_test, y_prob)) 

## EVALUEATE THE SAMPLING TECHNIQUES

In [17]:
print("Random Undersampling:")
evaluate_model(X_RUS, y_RUS, X_test, y_test)

print("\nRandom Oversampling:")
evaluate_model(X_ROS, y_ROS, X_test, y_test)

print("\nSMOTE:")
evaluate_model(X_Smote, y_Smote, X_test, y_test)

print("\nTomek Links:")
evaluate_model(X_TL, y_TL, X_test, y_test)

print("\nClass Weights:")
evaluate_model(X_train, y_train, X_test, y_test, class_weights_dict) 

Random Undersampling:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      8781
           1       1.00      1.00      1.00      2503

    accuracy                           1.00     11284
   macro avg       1.00      1.00      1.00     11284
weighted avg       1.00      1.00      1.00     11284

AUC: 0.9999787522937399

Random Oversampling:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      8781
           1       1.00      1.00      1.00      2503

    accuracy                           1.00     11284
   macro avg       1.00      1.00      1.00     11284
weighted avg       1.00      1.00      1.00     11284

AUC: 0.9997105853115198

SMOTE:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      8781
           1       1.00      1.00      1.00      2503

    accuracy                           1.00     11284
   macro avg       1.00     

## CONCLUSION
We got good AUC for all Models.